In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
%autosave 5

Autosaving every 5 seconds


In [2]:
def loadData(dataset,split=0):
    if(split == 0):
        print("In non split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51)
    else:
        print("In split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51,
                                                            validation_split=split,subset='both')

In [3]:
#from https://aakashgoel12.medium.com/how-to-add-user-defined-function-get-f1-score-in-keras-metrics-3013f979ce0d
def f1_score(y_true, y_pred): #taken from old keras source code 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [4]:
def main():
    train = loadData("training")
    dev,test = loadData("testing",.5)

    modelParameters = dict()

    choose = np.random.choice
    
    possibleConvFilterWidths = 2*np.arange(6)+1
    possibleConvStrides = np.arange(1,12)
    possibleConvFilters = np.arange(100)

    possiblePoolFilterWidths = np.arange(2,10)
    possiblePoolStrides = np.arange(1,10)

    possibleConvAndPoolLayers = np.arange(1,10)

    possibleNumberOfFCLayers = np.arange(50)
    possibleNumberOfNeuronsPerFCLayer = np.arange(50)

    possibleNumberOfEpochs = np.arange(100)
    
    bestDevF1 = 0
    
    for trial in range(1):
        convFilterWidth = choose(possibleConvFilterWidths)
        convStride = choose(possibleConvStrides)
        convFilters = choose(possibleConvFilters)

        poolFilterWidth = choose(possiblePoolFilterWidths) 
        poolStride = choose(possiblePoolStrides)

        convAndPoolLayers = choose(possibleConvAndPoolLayers)

        numberOfFCLayers = choose(possibleNumberOfFCLayers)
        numberOfNeuronsPerFCLayer = choose(possibleNumberOfNeuronsPerFCLayer)

        numberOfEpochs = choose(possibleNumberOfEpochs)
        
        dropoutRate = np.random.rand()
        
        model = keras.Sequential()
        
        for convAndPoolLayer in range(convAndPoolLayers):
            model.add(keras.layers.Conv2D(convFilters,(convFilterWidth,convFilterWidth),strides=(convStride,convStride), activation='relu',padding='same'))
            model.add(keras.layers.MaxPooling2D(pool_size=(poolFilterWidth, poolFilterWidth),
                                                strides=(poolStride,poolStride), padding='same'))
            
        model.add(keras.layers.Flatten())
        
        for layer in range(numberOfFCLayers):
            if layer == numberOfFCLayers - 1:
                model.add(keras.layers.Dense(1,activation='relu'))
            else:
                model.add(keras.layers.Dense(numberOfNeuronsPerFCLayer,activation='relu'))
            if layer != numberOfFCLayers - 1:
                model.add(keras.layers.Dropout(dropoutRate))            
    
        model.compile(optimizer="adam",loss='binary_crossentropy', metrics=[f1_score])
    
        model.fit(train,epochs=numberOfEpochs)

        model_path = f'../Models/fd_model_{trial}.h5'
        model.save(model_path)
        model_size = os.path.getsize(model_path) / (1024 * 1024)
        print('Model size: ', model_size, 'MB')
        
        print(model.metrics_names)
        print("train: ",model.evaluate(train))
        print("dev: ",model.evaluate(dev))
    
    

In [5]:
main()

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.
Epoch 1/72


2024-06-11 06:40:28.563432: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-06-11 06:40:28.563584: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2392]
	 [[{{node Placeholder/_0}}]]
2024-06-11 06:40:29.458116: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


75/75 [==============================] - 6s 61ms/step - loss: 2.6502 - f1_score: 0.0013
Epoch 2/72
75/75 [==============================] - 5s 60ms/step - loss: 1.7724 - f1_score: 0.0029
Epoch 3/72
75/75 [==============================] - 5s 61ms/step - loss: 1.5614 - f1_score: 0.0013
Epoch 4/72
75/75 [==============================] - 5s 61ms/step - loss: 1.4096 - f1_score: 0.0047
Epoch 5/72
75/75 [==============================] - 5s 62ms/step - loss: 1.3256 - f1_score: 0.0000e+00
Epoch 6/72
75/75 [==============================] - 5s 63ms/step - loss: 1.2134 - f1_score: 0.0163
Epoch 7/72
75/75 [==============================] - 5s 60ms/step - loss: 1.1000 - f1_score: 0.0801
Epoch 8/72
75/75 [==============================] - 5s 64ms/step - loss: 1.0494 - f1_score: 0.1208
Epoch 9/72
75/75 [==============================] - 5s 60ms/step - loss: 1.0161 - f1_score: 0.1044
Epoch 10/72
75/75 [==============================] - 5s 67ms/step - loss: 1.0019 - f1_score: 0.1473
Epoch 11/72
75/7

2024-06-11 06:46:42.288982: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [299]
	 [[{{node Placeholder/_0}}]]
2024-06-11 06:46:42.289123: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [299]
	 [[{{node Placeholder/_0}}]]


10/10 [==============================] - 1s 10ms/step - loss: 0.6947 - f1_score: 0.0000e+00
dev:  [0.694689929485321, 0.0]
